In [ ]:
import selenium as selenium

print("Project 1")

In [ ]:
#Trading strategy with moving average 
#TODO: Changing strategy to trade every 5 years (on a Wednesday?); shorten training data
#Total number of trades with current code: 26
#PSR: 0.055 (can improve)

class TradingStrategy(QCAlgorithm):
    #Keep track of highest price for the stock 
    highestSPYPrice = 0
    def Initialize(self):
        self.SetCash(100000)  # Set Strategy Cash
        self.averages = {}
        self.spy = self.AddEquity("SPY",Resolution.Daily)
        self.spy.SetDataNormalizationMode(DataNormalizationMode.Raw)


    def OnData(self, data):
         if self.Securities["SPY"].Close > self.highestSPYPrice:
             self.highestSPYPrice = self.Securities["SPY"].Close
        #Check the moving average of the stock to decide if trading or holding 
         self.SelectionFunction(self.spy)
            
    def SelectionFunction(self, stock):
        #Earliest data available for the 3 stocks
        #Change accordingly for different timeslots (start_date starting at 1998,1,1)
        start_date = datetime(1998, 1, 1)
        #Up to the start of 2022 (need to shorten)
        end_date = datetime(2022, 1, 1)
        symbol = stock.Symbol
        #History for stock with daily resolution
        history = self.History(symbol, start_date, end_date, Resolution.Daily)
        #The tutorials did something with the code below but equity doesnt have AdjustedPrice
        #self.averages[symbol].update(self.Time, stock.AdjustedPrice)
        self.averages[symbol] = SelectionData(history)
        self.Debug(str(history))
        if not self.Time.weekday() == 1:
            return
        #Decide if trade or hold
        if  self.averages[symbol].is_ready() and self.averages[symbol].fast > self.averages[symbol].slow:
            #Set asset holdings to 100% of symbol
            self.SetHoldings(symbol, 1)
        else: 
            #Liquidate assets
            self.Liquidate(symbol)
            #self.Liquidate(tag = "Liquidated")

#Moving average models
class SelectionData():
    def __init__(self, history):
        #Not sure if it should be 100 or 200 (I had both numbers in my notes)
        self.slow = ExponentialMovingAverage(200)
        self.fast = ExponentialMovingAverage(50)
        #4. Loop over the history data and update the indicators
        for bar in history.itertuples():
            self.fast.Update(bar.Index[1], bar.close)
            self.slow.Update(bar.Index[1], bar.close)

    def is_ready(self):
        return self.slow.IsReady and self.fast.IsReady

In [ ]:
# Trading strategy with Momentum Indicators

from AlgorithmImports import *

class MomentumBasedTacticalAllocation(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2013, 1, 1)
        self.SetEndDate(2018, 1, 1)
        self.SetCash(100000)

        self.apple = self.AddEquity("KO", Resolution.Daily)
        self.bnd = self.AddEquity("BND", Resolution.Daily)
        self.applemomentum = self.MOMP("KO", 50, Resolution.Daily)
        self.bondMomentum = self.MOMP("BND", 50, Resolution.Daily)
        self.SetBenchmark("KO")
        self.SetWarmUp(50)

    def OnData(self, data):

        # Don't place trades until our indicators are warmed up:
        if self.IsWarmingUp:
            return

        #1. If SPY has more upward momentum than BND, then we liquidate our holdings in BND and allocate 100% of our equity to SPY
        if self.applemomentum.Current.Value > self.bondMomentum.Current.Value:
            self.Liquidate("BND")
            self.SetHoldings("KO", 1)
        else:
            self.Liquidate("KO")
            self.SetHoldings("BND", 1)

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Crawling using Selenium 4
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# start a new Selenium session and navigate to the website
driver = webdriver.Chrome()
driver.get('https://arxiv.org/search/?query=BERT&searchtype=title&abstracts=show&order=-announced_date_first&size=50')

# wait for the search results to load
search_results = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, "//p[@class='title is-5 mathjax']"))
)

# extract titles from each search result
for result in search_results:
    title = result.text.strip()
    print(title)

print(len(search_results))

# close the browser window
driver.quit()

Coupling Artificial Neurons in BERT and Biological Neurons in the Human Brain
Indonesian Text-to-Image Synthesis with Sentence-BERT and FastGAN
Interpretable Bangla Sarcasm Detection using BERT and Explainable AI
Is BERT Blind? Exploring the Effect of Vision-and-Language Pretraining on Visual Language Understanding
Trained on 100 million words and still in shape: BERT meets British National Corpus
SmartBERT: A Promotion of Dynamic Early Exiting Mechanism for Accelerating BERT Inference
MEDBERT.de: A Comprehensive German BERT Model for the Medical Domain
Geolocation Predicting of Tweets Using BERT-Based Models
German BERT Model for Legal Named Entity Recognition
Can BERT Refrain from Forgetting on Sequential Tasks? A Probing Study
Argument Mining using BERT and Self-Attention based Embeddings
Using Auxiliary Tasks In Multimodal Fusion Of Wav2vec 2.0 And BERT For Multimodal Emotion Recognition
Mask-guided BERT for Few Shot Text Classification
Can ChatGPT Understand Too? A Comparative Stu

In [3]:
from selenium.webdriver.common.keys import Keys
import regex as re

In [6]:
print("Courses offered in Fall 2023")

# start a new Selenium session and navigate to the website
driver = webdriver.Chrome()
driver.get('https://atlas.emory.edu/')

# wait for the search bar to load
search_bar = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@id='crit-keyword']"))
)

# type 'CS' into the search bar and submit the search
search_bar.send_keys('CS')
search_bar.send_keys(Keys.RETURN)

search_results = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, "/html/body/main/div[2]/div/div[3]"))
)

# extract titles from each search result
for result in search_results:
    title = result.text.strip()

course_codes = re.findall(r"CS\s\d{3}", title)
print(course_codes)

# close the browser window
driver.quit()

Courses offered in Fall 2023
['CS 110', 'CS 170', 'CS 171', 'CS 211', 'CS 224', 'CS 253', 'CS 255', 'CS 312', 'CS 325', 'CS 326', 'CS 334', 'CS 350', 'CS 370', 'CS 377', 'CS 385', 'CS 485', 'CS 485', 'CS 526', 'CS 534', 'CS 551', 'CS 557', 'CS 571', 'CS 584', 'CS 584', 'CS 700', 'CS 799']
